# Predicciones utilizando el mejor modelo
## 1. Importación de librerías, establecimiento de directorio de trabajo y carga de datos

In [1]:
# librerias principales para uso y visualización de datos
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings  
warnings.filterwarnings('ignore')

# normalizar
from sklearn.preprocessing import StandardScaler

# cargar el modelo
import pickle

os.chdir("C:/Users/aleja/OneDrive - Universidad Politécnica de Madrid/Universidad/Asignaturas/Cuarto/Trabajo de Fin de Grado")

In [2]:
# datos csv
accesos = pd.read_csv("Resultados/Accesos_Calidad_Vida.csv", index_col="Unnamed: 0")

# modelo
regressor = pickle.load(open('Modelos/XGBoost.pkl', 'rb'))

## 2. Construcción de la matriz de características (X) y el vector variable dependiente (y)

In [3]:
X = accesos.iloc[:, :-1].values
y = accesos.iloc[:, -1].values

## 3. Estandarizar X

In [4]:
sc = StandardScaler()
X = sc.fit_transform(X)

## 4. Predecir valores

In [5]:
y_pred = regressor.predict(X)

## 5. Cálculo de métricas de la predicción

In [6]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# R2score
print('R2score:', r2_score(y, y_pred))

# Mean Absolute Error
print('MAE: \t', mean_absolute_error(y, y_pred))

# Mean Squared Error
print('MSE: \t', mean_squared_error(y, y_pred))

# Root Mean Squared Error
print("RMSE: ", mean_squared_error(y, y_pred, squared=False))

R2score: 0.996781449645687
MAE: 	 0.2790320353591686
MSE: 	 2.42971607534776
RMSE:  1.558754655277013


## 6. Guardar resultados
### 6.1. En Dataframe

In [7]:
X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y_pred)
df = pd.concat([X_df, y_df], axis=1)
df.set_axis(['latitud', 'longitud', 'edad', 'calidad_vida'], axis='columns', inplace=True)
df.head()

,latitud,longitud,edad,calidad_vida
0,-1.149122,1.739274,-1.362061,22.693689
1,1.963275,-1.949366,-1.317309,21.209745
2,1.965770,-1.842890,-1.272556,21.516243
3,1.774915,-1.837662,-1.317309,12.038342
4,1.904349,-1.855992,-1.183051,20.745489


In [8]:
df.to_csv("Predicciones/XGBoost.csv")

### 6.2. En GeoDataframe

In [9]:
georf = gpd.GeoDataFrame(df.drop(['latitud', 'longitud'], axis=1), crs="EPSG:4326", geometry=gpd.points_from_xy(df['longitud'], df['latitud']))
georf.head()

,edad,calidad_vida,geometry
0,-1.362061,22.693689,POINT (1.73927 -1.14912)
1,-1.317309,21.209745,POINT (-1.94937 1.96328)
2,-1.272556,21.516243,POINT (-1.84289 1.96577)
3,-1.317309,12.038342,POINT (-1.83766 1.77491)
4,-1.183051,20.745489,POINT (-1.85599 1.90435)


In [10]:
georf.to_file("Predicciones/XGBoost.shp")